In [22]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

file_name = 'msamd_14454.csv'
dataframe = pd.read_csv(file_name, low_memory=False)

In [23]:
# column_names = dataframe.columns.tolist()
# for column in column_names:
#     print(column)

In [24]:
# # grouping by loan_type and derived_race and counting the occurrences
# race_distribution = dataframe.groupby(['loan_type', 'derived_race']).size().reset_index(name='count')
# print(race_distribution)

In [25]:
fig = make_subplots(rows=1, cols=1)
# loan types
loan_types = [1, 2, 3, 4]

# iterate over each loan type and add the corresponding button, trace
for loan_type in loan_types:
    filtered_data = dataframe[(dataframe['loan_type'] == loan_type) &
                              (~dataframe['derived_race'].isin(['2 or more minority races',
                                                                'Free Form Text Only',
                                                                'Race Not Available',
                                                                'Joint']))]

    race_distribution = filtered_data['derived_race'].value_counts().reset_index()

    fig.add_trace(go.Bar(y=race_distribution['index'], x=race_distribution['derived_race'], orientation='h',
                         name=f'Loan Type {loan_type}'))

# update layout
fig.update_layout(height=600, showlegend=False, yaxis=dict(title='Race'), xaxis=dict(title='Count'),
                  title='Racial Distribution by Loan Type')
# define loan type with descriptions
loan_types_def = {
    1: "Conventional (not FHA, VA, RHS, or FSA)",
    2: "FHA Insured",
    3: "VA Guaranteed",
    4: "RHS or FSA Guaranteed"
}
# buttons for loan types
buttons = []
for loan_type in loan_types:
    visible = [False] * len(loan_types)
    visible[loan_type - 1] = True
    buttons.append(dict(label=f'{loan_types_def[loan_type]}', method='update',
                        args=[{'visible': visible}, {'title': f'Racial Distribution for {loan_types_def[loan_type]}'}]))

fig.update_layout(updatemenus=[dict(buttons=buttons, type="buttons", showactive=True, x=0.1, y=-0.15)])
fig.show()


In [26]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=1)

loan_types = [1, 2, 3, 4]

for loan_type in loan_types:
    filtered_data = dataframe[dataframe['loan_type'] == loan_type]

    ethnicity_distribution = filtered_data['derived_ethnicity'].value_counts().reset_index()

    fig.add_trace(go.Bar(y=ethnicity_distribution['index'], x=ethnicity_distribution['derived_ethnicity'],
                         orientation='h', name=f'Loan Type {loan_type}'))

fig.update_layout(height=600, showlegend=False, yaxis=dict(title='Ethnicity'), xaxis=dict(title='Count'),
                  title='Ethnicity Distribution by Loan Type')

loan_types_def = {
    1: "Conventional (not FHA, VA, RHS, or FSA)",
    2: "FHA Insured",
    3: "VA Guaranteed",
    4: "RHS or FSA Guaranteed"
}

buttons = []
for loan_type in loan_types:
    visible = [False] * len(loan_types)
    visible[loan_type - 1] = True
    buttons.append(dict(label=f'{loan_types_def[loan_type]}', method='update',
                        args=[{'visible': visible}, {'title': f'Ethnicity Distribution for {loan_types_def[loan_type]}'}]))

fig.update_layout(updatemenus=[dict(buttons=buttons, type="buttons", showactive=True, x=0.1, y=-0.15)])
fig.show()


I will be creating a new column from entries in 'action_taken' to summarize if the applicants loan is approved, here is what the values of action_taken mean:

- **1: Loan originated**
- **2: Application approved but not accepted**
- **3: Application denied**
- **4: Application withdrawn by applicant**
- **5: File closed for incompleteness**
- **6: Purchased loan**
- **7: Preapproval request denied**
- **8: Preapproval request approved but not accepted**

Here are the values of action_taken that mean the loan is approved, and these are the entries I will use for my new column 'approved' (values 0 and 1. 1 for approved and 0 for not):

- **1: Loan originated**
- **2: Application approved but not accepted**
- **6: Purchased loan**
- **8: Preapproval request approved but not accepted**

These values represent different stages of loan approval or acceptance. 


In [29]:
# list of values for the 'approved' column
approved_values = [1 if action in [1, 2, 6, 8] else 0 for action in dataframe['action_taken']]
# add the 'approved' column to the dataframe
dataframe['approved'] = approved_values

In [51]:
import pandas as pd
import plotly.express as px
import scipy.stats as stats

# Define the excluded derived_race categories
excluded_categories = ['2 or more minority races', 'Free Form Text Only', 'Race Not Available', 'Joint']

# ilter the dataframe for the convention loan type and exclude the specified derived_race categories
filtered_data = dataframe[(dataframe['loan_type'] == 1) &
                          (~dataframe['derived_race'].isin(excluded_categories))]

# approval rates calculation as proportions by derived_race
approval_rates = filtered_data.groupby('derived_race')['approved'].mean()

# the chi-Squared test
expected_rate = filtered_data['approved'].mean()
observed = filtered_data.groupby('derived_race')['approved'].sum()
expected = expected_rate * filtered_data.groupby('derived_race').size()
chi2, p_value = stats.chisquare(observed, f_exp=expected)

# null hypothesis
print("Null Hypothesis: There is no association between the 'derived_race' and the approval of loans for loan type 1.")

# p-value of the Chi-Squared test
print("Chi-Squared Test Results:")
print(f"P-value: {p_value}")

# significance level
alpha = 0.05

# interpretation of the p-value
if p_value < alpha:
    print("The p-value is less than the significance level of 0.05.")
    print("There is evidence to reject the null hypothesis.")
else:
    print("The p-value is greater than or equal to the significance level of 0.05.")
    print("There is not enough evidence to reject the null hypothesis.")

# bar plot of the approval rates
fig = px.bar(approval_rates, x=approval_rates.index, y=approval_rates.values,
             labels={'x': 'Derived Race', 'y': 'Approval Rate'},
             title='Approval Rates (Proportions) by Derived Race for Loan Type 1')
fig.show()


Null Hypothesis: There is no association between the 'derived_race' and the approval of loans for loan type 1.
Chi-Squared Test Results:
P-value: 5.522737739181438e-93
The p-value is less than the significance level of 0.05.
There is evidence to reject the null hypothesis.
